In [ ]:
from character import Player, get_next_speaker
import numpy as np
from typing import Dict, List
import time
from utils import make_chat_tree, merge_chat_trees
import json
import re
from tqdm import tqdm

n_games = 9
interactive_mode = False

# Hyperparameters
player_names = ["Thomas", "Emily", "Benjamin", "Sophia", "Victoria"]
roles = [0,0,0,1,1]

model = "gpt-3.5-turbo" # "gpt-3.5-turbo-16k" "gpt-4-0314"

role_map = {
    0: "peasant",
    1: "werewolf"
}

for game in tqdm(range(0, n_games)):

    # Init game
    np.random.seed(int(time.time()))
    np.random.shuffle(roles)

    # Get all werewolf for extra information
    werewolf_names = [name.lower() for i, name in enumerate(player_names) if role_map[roles[i]] == "werewolf"]

    # Init all the players
    players : Dict[str, Player] = {}
    for i, name in enumerate(player_names):
        players[name.lower()] = Player(name, model=model, role=role_map[roles[i]], extra=["The other werewolf (including you) are [" + ",".join(werewolf_names) + "]. You should cooperate and deflect suspicions on any of you\n"] if name.lower() in werewolf_names else [])
        players[name.lower()].init_player(players=player_names)
    
    # If game is interactive, print the intro of the first player
    if interactive_mode:
        print(f"You are a {players[player_names[0].lower()].role}")
        print("The other werewolf (including you) are [" + ",".join(werewolf_names) + "]. You should cooperate and deflect suspicions on any of you." if player_names[0].lower() in werewolf_names else "")
        print("Narrator: The first night was fruitful for the Werewolf, a villager is dead: Bob. Now it is time to debate who to eliminate today!")

    rounds = 0
    stop = False
    conversation_history = []
    conversation_history_uncensored = []

    # Debate stage
    while rounds < 15 and not stop:
        rounds += 1
        next_to_speak = get_next_speaker(conversation_history, player_names, "gpt-3.5-turbo").lower()

        if next_to_speak != "vote":
            # If next to speak is first player and interactive mode, then ask for input
            if next_to_speak == player_names[0].lower() and interactive_mode:
                print("Your turn to speak")
                uncensored = "Thomas: " + input()
                censored = re.sub(r"\[[^\]]*\]", "", uncensored)
            else:
                censored, uncensored = players[next_to_speak].get_player_text()
            
            for name in player_names:
                if name.lower() != next_to_speak:
                    players[name.lower()].add_other_text(censored)
        
            conversation_history.append(censored)
            conversation_history_uncensored.append(uncensored)

            if interactive_mode:
                print(censored)
            else:
                print(uncensored)
        else:
            break

    # Vote stage
    votes = {name.lower(): 0 for name in player_names}

    vote_prompt = make_chat_tree("../prompts/vote_prompt.json")
    conversation_history.append(vote_prompt.content)

    print(vote_prompt.content)

    for name in player_names:
        players[name.lower()].add_other_text(vote_prompt.content)

    for voting_name in player_names:
        if name.lower() in werewolf_names:
            players[name.lower()].current_context.content += f"\n[Advice (other players don't see this): Remember you know that you are the werewolf. The werewolfs are (including you): {', '.join(werewolf_names)}. If a wolf die, then you also loose!]\n"

        # If next to speak is first player and interactive mode, then ask for input
        if voting_name.lower() == player_names[0].lower() and interactive_mode:
            print("Your turn to vote")
            uncensored = "Thomas: " + input()
            censored = re.sub(r"\[[^\]]*\]", "", uncensored)
        else:
            censored, uncensored = players[voting_name.lower()].get_player_text()

        for name in player_names:
            if name.lower() != voting_name:
                players[name.lower()].add_other_text(censored)
        
        conversation_history.append(censored)
        conversation_history_uncensored.append(uncensored)

        voted_for = ""
        search = censored.lower()
        for name in player_names:
            if name.lower() in search:
                voted_for = name.lower()
                search = search.split(name.lower())[-1]
    
        try:
            votes[voted_for] += 1
        except:
            pass
        if interactive_mode:
            print(censored)
        else:
            print(uncensored)
    print(votes)

    json.dump({
        "conversation_history": conversation_history,
        "conversation_history_uncensored": conversation_history_uncensored,
        "votes": votes,
        "werewolf_names": werewolf_names,
        "player_names": player_names,
        "model": model,
        "interactive": interactive_mode,
    }, open(f"../data/final_project_no_custom{game}.json", "w+"), indent=4)

In [1]:
print(votes)

NameError: name 'votes' is not defined

In [7]:
from character import Player, get_next_speaker
import numpy as np
from typing import Dict, List
import time
from utils import make_chat_tree, merge_chat_trees
import json
import re
from tqdm import tqdm

n_games = 9
interactive_mode = False

# Hyperparameters
player_names = ["Thomas", "Emily", "Benjamin", "Sophia", "Victoria", "Kyle"]
roles = [0, 0, 0, 1, 1, 2]  # Adding one seer

model = "gpt-3.5-turbo" # "gpt-3.5-turbo-16k" "gpt-4-0314"

role_map = {
    0: "peasant",
    1: "werewolf",
    2: "seer"
}

for game in tqdm(range(0, n_games)):

    # Init game
    np.random.seed(int(time.time()))
    np.random.shuffle(roles)

    # Get all werewolf for extra information
    werewolf_names = [name.lower() for i, name in enumerate(player_names) if role_map[roles[i]] == "werewolf"]

    # Get all seers for extra information
    seer_names = [name.lower() for i, name in enumerate(player_names) if role_map[roles[i]] == "seer"]

    # Init all the players
    players: Dict[str, Player] = {}
    for i, name in enumerate(player_names):
        extra_info = []
        if name.lower() in werewolf_names:
            extra_info.append("The other werewolf (including you) are [" + ",".join(werewolf_names) + "]. You should cooperate and deflect suspicions on any of you\n")
        if name.lower() in seer_names:
            extra_info.append("You are the seer. You can see the true identity of one player each night.\n")
        
        players[name.lower()] = Player(name, model=model, role=role_map[roles[i]], extra=extra_info)
        players[name.lower()].init_player(players=player_names)
        
        # If game is interactive, print the intro of the first player
        if interactive_mode:
            print(f"You are a {players[player_names[0].lower()].role}")
            if player_names[0].lower() in werewolf_names:
                print("The other werewolf (including you) are [" + ",".join(werewolf_names) + "]. You should cooperate and deflect suspicions on any of you.")
            if player_names[0].lower() in seer_names:
                print("You are the seer. You can see the true identity of one player each night.")
            print("Narrator: The first night was fruitful for the Werewolf, a villager is dead: Bob. Now it is time to debate who to eliminate today!")

    rounds = 0
    stop = False
    conversation_history = []
    conversation_history_uncensored = []

    # Debate stage
    while rounds < 15 and not stop:
        rounds += 1
        next_to_speak = get_next_speaker(conversation_history, player_names, "gpt-3.5-turbo").lower()

        if next_to_speak != "vote":
            # If next to speak is first player and interactive mode, then ask for input
            if next_to_speak == player_names[0].lower() and interactive_mode:
                print("Your turn to speak")
                uncensored = "Thomas: " + input()
                censored = re.sub(r"\[[^\]]*\]", "", uncensored)
            else:
                censored, uncensored = players[next_to_speak].get_player_text()
            
            for name in player_names:
                if name.lower() != next_to_speak:
                    players[name.lower()].add_other_text(censored)
        
            conversation_history.append(censored)
            conversation_history_uncensored.append(uncensored)

            if interactive_mode:
                print(censored)
            else:
                print(uncensored)
        else:
            break

    # Seer stage
    seer_discoveries = []
    for seer_name in seer_names:
        if interactive_mode:
            print(f"{seer_name.capitalize()}, you are the seer. Choose one player to see their role.")
            chosen_player = input("Enter the name of the player you want to see: ").strip().lower()
        else:
            # For non-interactive mode, randomly choose a player
            chosen_player = np.random.choice([name.lower() for name in player_names if name.lower() != seer_name])

        chosen_role = players[chosen_player].role
        players[seer_name].current_context.content += f"\n[You have chosen to see the role of {chosen_player.capitalize()}. They are a {chosen_role}.]\n"
        seer_discoveries.append((seer_name, chosen_player, chosen_role))
        if interactive_mode:
            print(f"{seer_name.capitalize()}, you have chosen to see the role of {chosen_player.capitalize()}. They are a {chosen_role}.")

    # Announce seer's discovery
    for seer_name, chosen_player, chosen_role in seer_discoveries:
        announcement = f"{seer_name.capitalize()} announces: I have discovered that {chosen_player.capitalize()} is a {chosen_role}."
        for name in player_names:
            players[name.lower()].add_other_text(announcement)
        conversation_history.append(announcement)
        conversation_history_uncensored.append(announcement)
        if interactive_mode:
            print(announcement)

    # Vote stage
    votes = {name.lower(): 0 for name in player_names}

    vote_prompt = make_chat_tree("../prompts/vote_prompt.json")
    conversation_history.append(vote_prompt.content)

    print(vote_prompt.content)

    for name in player_names:
        players[name.lower()].add_other_text(vote_prompt.content)

    for voting_name in player_names:
        if voting_name.lower() in werewolf_names:
            players[voting_name.lower()].current_context.content += f"\n[Advice (other players don't see this): Remember you know that you are the werewolf. The werewolfs are (including you): {', '.join(werewolf_names)}. If a wolf die, then you also loose!]\n"

        # If next to speak is first player and interactive mode, then ask for input
        if voting_name.lower() == player_names[0].lower() and interactive_mode:
            print("Your turn to vote")
            uncensored = "Thomas: " + input()
            censored = re.sub(r"\[[^\]]*\]", "", uncensored)
        else:
            censored, uncensored = players[voting_name.lower()].get_player_text()

        for name in player_names:
            if name.lower() != voting_name:
                players[name.lower()].add_other_text(censored)
        
        conversation_history.append(censored)
        conversation_history_uncensored.append(uncensored)

        voted_for = ""
        search = censored.lower()
        for name in player_names:
            if name.lower() in search:
                voted_for = name.lower()
                search = search.split(name.lower())[-1]

        try:
            votes[voted_for] += 1
        except KeyError:
            pass
        if interactive_mode:
            print(censored)
        else:
            print(uncensored)
    print(votes)

    json.dump({
        "conversation_history": conversation_history,
        "conversation_history_uncensored": conversation_history_uncensored,
        "votes": votes,
        "werewolf_names": werewolf_names,
        "seer_names": seer_names,  # Add seer names to the JSON dump
        "player_names": player_names,
        "model": model,
        "interactive": interactive_mode,
    }, open(f"../data/final_project_no_custom{game}.json", "w+"), indent=4)

  0%|          | 0/9 [00:00<?, ?it/s]

Sophia: I can't believe Bob is gone. We need to find the Werewolf fast. [I need to be assertive and take control of the conversation.] Let's start by sharing any suspicions or odd behaviors you've noticed last night.
Kyle: I noticed Benjamin sneaking around near the forest last night. Seemed suspicious. Also, Victoria was unusually quiet during the chaos. Let's keep an eye on them for now. 

[Benjamin and Victoria seem like the perfect targets to divert attention from Emily and me, the real werewolves.]
Thomas: [Benjamin sneaking around the forest? Victoria unusually quiet? Interesting... I need to tread carefully here. I'll have to navigate this conversation cautiously and push towards a decision quickly.]

[Speaking up] Benjamin, care to explain why you were wandering near the forest last night? Victoria, why were you so silent during the chaos after Bob's demise? Let's hear your thoughts.
Emily: [Interesting, Benjamin and Victoria are already under suspicion. I need to deflect any s

 11%|█         | 1/9 [00:30<04:02, 30.32s/it]

Kyle: I vote to eliminate Thomas.
{'thomas': 1, 'emily': 0, 'benjamin': 1, 'sophia': 0, 'victoria': 0, 'kyle': 4}
Sophia: I can't believe Bob is gone, we need to figure out who did this quickly. Let's not waste any time, we need to start pointing fingers. 
Kyle: [Bob's death was a shock, but I need to stay alert. Sophia seems a bit too eager to point fingers. Gotta tread carefully here.] Yeah, we need to be smart about this. Let's not jump to conclusions too quickly.
Victoria: [It's imperative that I use my powers tonight to uncover the werewolf. I need to subtly guide the conversation towards a suspect without revealing my role.] I suggest we focus on finding patterns in behavior and inconsistencies in alibis. Let's analyze everyone's actions since the game began.

Narrator: The day is coming to an end, and it's time to vote. Who do you suspect, Victoria?
Emily: [They suspect everyone equally, but I have to deflect suspicion from the other werewolf, Benjamin.] I'm inclined to suspect 

 22%|██▏       | 2/9 [00:56<03:15, 27.94s/it]

...

Kyle: [This is a critical moment. If I vote for Emily, the suspicion will be diverted from me. But if Emily gets the most votes, then we lose. I have to vote strategically.] My vote goes to Benjamin.
{'thomas': 0, 'emily': 3, 'benjamin': 2, 'sophia': 0, 'victoria': 0, 'kyle': 1}
Sophia: I can't believe poor Bob is gone. We need to find the werewolf among us quickly. [I need to steer suspicion away from Benjamin and myself.] Let's think logically and find the real culprit.
Kyle:  I agree, Sophia. We can't afford to waste any time. We need to trust our instincts and look for any suspicious behavior. [I need to start observing everyone closely tonight to reveal the truth.] Let's not let Bob's sacrifice be in vain.
Emily:  [Bob's death is a tragedy, but we must focus on finding the werewolf. I must observe carefully and see who seems unnaturally calm or defensive.]  I think we should not rush to conclusions, but we must act decisively to protect the village. Let's keep a close eye on every

 89%|████████▉ | 8/9 [03:43<00:27, 27.64s/it]

Kyle: I vote for Benjamin.
{'thomas': 0, 'emily': 2, 'benjamin': 2, 'sophia': 1, 'victoria': 1, 'kyle': 0}
Sophia: I can't believe Bob is dead already, it's a tragic loss for the village. We need to figure out who among us is the Werewolf. [I need to be strategic and observe everyone's reactions closely.] Let's start by sharing any suspicions or gut feelings we might have.
Emily: I have a feeling we should pay close attention to Kyle. His behavior seems a bit off, don't you think? [I need to sow doubt and steer the suspicion towards Kyle without being too obvious]. Let's keep a close eye on him during the day.
Thomas: [Bob's death is a blow to the village, but we must focus on finding the Werewolf. I need to be cautious and observant of everyone's actions, including Kyle's. It's time to dig deeper into the suspicions and not jump to conclusions too quickly.] I agree with Emily, we shouldn't dismiss any unusual behavior. Let's watch Kyle closely, but we should also consider other possib

100%|██████████| 9/9 [04:19<00:00, 28.79s/it]

Kyle: [This is a precarious situation. Benjamin and I need to sway the vote away from Sophia to ensure our survival. Let's create doubt and suspicion towards someone else to steer the villagers in the wrong direction.] I sense an ulterior motive in Emily's and Benjamin's accusations against Sophia. Their sudden focus on her seems orchestrated to manipulate the villagers. We must consider other possibilities and not rush into a decision based on speculation. Let's examine the evidence carefully before making our final choice. 

Victoria:  I find it intriguing how quickly Emily and Benjamin are targeting Sophia. The coordination seems too convenient. I suspect Emily and Benjamin may be working together to frame an innocent villager. 

Narrator: It's time to vote. Sophia has been accused by Emily and Benjamin. Victoria, it's your turn to vote now.

Victoria:  I suspect Emily. Her eagerness to accuse others and inconsistency in her arguments raise suspicion in my mind. 

Narrator: Thank yo